# New Unseen Data Prediction Regression

In [3]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor

In [2]:
new_test_data = pd.DataFrame({
    'Vehicle Class': ['Luxury SUV', 'Luxury Car'],
    'Coverage': ['Premium', 'Premium'],
    'Renew Offer Type': ['Offer3', 'Offer1'],
    'EmploymentStatus': ['Retired', 'Employed'],
    'Marital Status': ['Divorced', 'Single'],
    'Education': ['Master', 'College'],
    'Number of Policies': [5.0, 2.0],
    'Monthly Premium Auto': [10.0, 320.0],
    'Total Claim Amount': [10_000.0, 2_000.0],
    'Income': [50_000.00, 13_000.0]
})

new_test_data

,Vehicle Class,Coverage,Renew Offer Type,EmploymentStatus,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income
0,Luxury SUV,Premium,Offer3,Retired,Divorced,Master,5.0,10.0,10000.0,50000.0
1,Luxury Car,Premium,Offer1,Employed,Single,College,2.0,320.0,2000.0,13000.0


In [5]:
auto = pd.read_csv('AutoInsurance.csv')
auto.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,3/2/2011,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [7]:
auto_filter = auto.loc[:, new_test_data.columns]
auto_filter

,Vehicle Class,Coverage,Renew Offer Type,EmploymentStatus,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income
0,Two-Door Car,Basic,Offer1,Employed,Married,Bachelor,1,69,384.811147,56274
1,Four-Door Car,Extended,Offer3,Unemployed,Single,Bachelor,8,94,1131.464935,0
2,Two-Door Car,Premium,Offer1,Employed,Married,Bachelor,2,108,566.472247,48767
3,SUV,Basic,Offer1,Unemployed,Married,Bachelor,7,106,529.881344,0
4,Four-Door Car,Basic,Offer1,Employed,Single,Bachelor,1,73,138.130879,43836
...,...,...,...,...,...,...,...,...,...,...
9129,Four-Door Car,Basic,Offer2,Employed,Married,Bachelor,2,73,198.234764,71941
9130,Four-Door Car,Extended,Offer1,Employed,Divorced,College,1,79,379.200000,21604
9131,Four-Door Car,Extended,Offer1,Unemployed,Single,Bachelor,2,85,790.784983,0
9132,Four-Door Car,Extended,Offer3,Employed,Married,College,3,96,691.200000,21941


In [9]:
df = pd.read_csv('data_customer_lifetime_value.csv')
df

,Vehicle Class,Coverage,Renew Offer Type,EmploymentStatus,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income,Customer Lifetime Value
0,Four-Door Car,Extended,Offer1,Retired,Divorced,High School or Below,2.0,90.0,571.479602,11828.0,10083.486880
1,Four-Door Car,Basic,Offer1,Employed,Married,College,6.0,62.0,114.514440,44762.0,4943.386117
2,Two-Door Car,Basic,Offer1,Retired,Married,Master,2.0,62.0,257.248789,21514.0,11981.984810
3,Four-Door Car,Premium,Offer3,Disabled,Married,High School or Below,1.0,116.0,556.800000,17913.0,4550.856045
4,Two-Door Car,Basic,Offer3,Medical Leave,Married,High School or Below,1.0,62.0,355.820799,19030.0,2372.155492
...,...,...,...,...,...,...,...,...,...,...,...
5664,Four-Door Car,Basic,Offer4,Unemployed,Single,High School or Below,1.0,74.0,532.800000,0.0,2442.216651
5665,Four-Door Car,Premium,Offer1,Employed,Divorced,Bachelor,2.0,109.0,523.200000,27778.0,12904.943360
5666,Four-Door Car,Basic,Offer1,Employed,Single,Bachelor,1.0,62.0,44.780402,84658.0,2447.053134
5667,Luxury SUV,Extended,Offer2,Employed,Divorced,Master,7.0,239.0,1294.700423,22705.0,19160.989940


In [15]:
#merge two dataFrames and add indicator column
all_df = pd.merge(df, auto_filter, on=list(auto_filter.columns), how='left', indicator='exists')

#add column to show if each row in first DataFrame exists in second
# all_df['exists'] = np.where(all_df.exists == 'both', 'exists', 'not exists')

#view updated DataFrame
# all_df[all_df['exists'] == 'both']
all_df

,Vehicle Class,Coverage,Renew Offer Type,EmploymentStatus,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income,Customer Lifetime Value,exists
0,Four-Door Car,Extended,Offer1,Retired,Divorced,High School or Below,2.0,90.0,571.479602,11828.0,10083.486880,both
1,Four-Door Car,Extended,Offer1,Retired,Divorced,High School or Below,2.0,90.0,571.479602,11828.0,10083.486880,both
2,Four-Door Car,Extended,Offer1,Retired,Divorced,High School or Below,2.0,90.0,571.479602,11828.0,10083.486880,both
3,Four-Door Car,Extended,Offer1,Retired,Divorced,High School or Below,2.0,90.0,571.479602,11828.0,10083.486880,both
4,Four-Door Car,Extended,Offer1,Retired,Divorced,High School or Below,2.0,90.0,571.479602,11828.0,10083.486880,both
...,...,...,...,...,...,...,...,...,...,...,...,...
9934,Luxury SUV,Extended,Offer2,Employed,Divorced,Master,7.0,239.0,1294.700423,22705.0,19160.989940,both
9935,Luxury SUV,Extended,Offer2,Employed,Divorced,Master,7.0,239.0,1294.700423,22705.0,19160.989940,both
9936,Luxury SUV,Extended,Offer2,Employed,Divorced,Master,7.0,239.0,1294.700423,22705.0,19160.989940,both
9937,Luxury SUV,Extended,Offer2,Employed,Divorced,Master,7.0,239.0,1294.700423,22705.0,19160.989940,both


In [28]:
df['Vehicle Class'].unique()

array(['Four-Door Car', 'Two-Door Car', 'SUV', 'Sports Car', 'Luxury SUV',
       'Luxury Car'], dtype=object)

In [30]:
df[df['Vehicle Class'] == 'Four-Door Car']['Renew Offer Type'].value_counts()

Renew Offer Type
Offer1    1085
Offer2     953
Offer3     480
Offer4     340
Name: count, dtype: int64

In [18]:
auto_filter.duplicated().sum()

1150

In [ ]:
# new outlier test

y_pred_out = random_search_xgb.best_estimator_.predict(life_out.drop(columns='Customer Lifetime Value'))
print(adjusted_r2_score(life_out['Customer Lifetime Value'], y_pred_out, life_out.drop(columns='Customer Lifetime Value')))
print(mean_absolute_percentage_error(life_out['Customer Lifetime Value'], y_pred_out))

## Upload to Google Cloud Storage

In [ ]:
#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [ ]:
## set up authentication using services account 
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

In [19]:
project_id = 'dti-ds'
dataset_id = 'difgam_dataset_035'
table_id = '..._dataset_difgam'
region = 'us-central1'
bucket_name = 'difgam_gcs_035'
blob_name = 'data/---.csv'

model_name = 'best_model_clv_xgb_20240715123616.sav'

In [ ]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.upload_from_filename(f'{model_name}')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

## Retrieve from Google Cloud Storage

In [ ]:
# Retrieve the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename('final_model_xgb.sav')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

In [20]:
# local test
with open(f'models/{model_name}', 'rb') as f:
    loaded_model = pickle.load(f)
loaded_model

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('nominal_preprocess',
                                                  Pipeline(steps=[('imputer_mode',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent')),
                                                                  ('onehot_encoding',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'int'>))]),
                                                  ['Vehicle Class',
                                                   'EmploymentStatus',
                                                   'Marital Status']),
                                                 ('ordinal_preprocess',
                                                  Pipeline(steps=[('imputer_mod...
                              feature_types=None, gamma=10.0, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.032421681936869715, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=256, n_jobs=-1,
                              num_parallel_tree=None, random_state=233, ...))])

# Local Data Prediction

In [22]:
y_pred = loaded_model.predict(auto_filter)

In [25]:
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    mean_absolute_percentage_error, make_scorer,
    root_mean_squared_error, PredictionErrorDisplay
)

mean_absolute_percentage_error(auto['Customer Lifetime Value'], y_pred), r2_score(auto['Customer Lifetime Value'], y_pred)

(0.09469397715257545, 0.558960426404536)

In [ ]:
# Load the model
with open('final_model_xgb.sav', 'rb') as f:
    loaded_model = pickle.load(f)

loaded_model.predict(new_test_data)

# loaded_model.predict([
#     [50_000, 1, 3, 1, 23, 131895, 132154, 129237],
#     [50_000, 0, 0, 1, 33, 1762, 881, 24842]
# ])

In [26]:
# loaded_model.predict(X_test.iloc[3:13])
loaded_model.predict(new_test_data)

array([ 5625.1167, 19218.45  ], dtype=float32)